Here is an R code which samples the posterior from the signal/background problem using the Metropolis algorithm. It first simulates a set of data, then samples the posterior, then produces plots of the posterior. Note: to get it to work, download it to the same directory containing metropolis.R.

In [1]:
##### CBJ Jan. 2017
##### The signal/background estimation problem with posterior sampled by Metropolis

source("../PBI_code/metropolis.R")

# Define function to return true signal at position x (generative model)
signal <- function(x, a, b, x0, w, t) {
  t*(a*exp(-(x-x0)^2/(2*w^2)) + b)
}

# Define function to return log (base 10) posterior over theta=(a,b).
# Prior on a and b: P(a,b) = const if a>0 and b>0, = 0 otherwise.
# Likelihood for one point is Poisson with mean d(x), so total 
# likelihood is their product. Unnormalized posterior is product of these.
# d and x are equal length vectors (or scalars). The rest are scalars.
logupost <- function(theta, d, x, x0, w, t) {
  a <- theta[1]
  b <- theta[2]
  if(a<0 || b <0) {return(-Inf)} # the effect of the prior
  logupost <- sum(dpois(d, lambda=signal(x, a, b, x0, w, t), log=TRUE))
  return(c(0, logupost)/log(10))
}

# Set model parameters (true and fixed)
x0    <- 0 # centre of peak
w     <- 1 # sd of peak
atrue <- 2 # amplitude
btrue <- 1 # background
t     <- 5 # scale factor (exposure time -> sets SNR)

# Simulate some data (by drawing from the likelihood)
set.seed(205)
xdat  <- seq(from=-7*w, to=7*w, by=0.5*w)
strue <- signal(xdat, atrue, btrue, x0, w, t)
ddat  <- rpois(length(strue), strue)

# Sample posterior with Metropolis; parameters = (a,b)

sampleCov <- diag(c(0.1, 0.1)^2)
# Set starting point
thetaInit <- c(1, 1)
# Run the MCMC to get samples from the posterior PDF
set.seed(100)
allSamp <- metrop(func=logupost, thetaInit=thetaInit, Nburnin=0, 
                  Nsamp=1e5, sampleCov=sampleCov, verbose=1e3, demo=FALSE,
                  d=ddat, x=xdat, x0=x0, w=w, t=t)
thinSel  <- seq(from=1, to=nrow(allSamp), by=25) # thin
postSamp <- allSamp[thinSel,]

# Plot MCMC chains and use density estimation to plot 1D posterior PDFs.
# We don't need to do any explicit marginalization to get the 1D PDFs.
pdf("signal_background_mcmc.pdf", width=8, height=7)
par(mfrow=c(3,2), mar=c(3.0,3.5,0.5,0.5), oma=0.5*c(1,1,1,1), 
    mgp=c(1.8,0.6,0), cex=0.9)
parname <- c("a", "b")
for(j in 3:4) { # columns of postSamp
  plot(1:nrow(postSamp), postSamp[,j], type="l",
       xlab="iteration", ylab=parname[j-2])
  postDen <- density(postSamp[,j], n=2^10)
  plot(postDen$x, postDen$y, type="l", lwd=1.5, yaxs="i", 
       ylim=1.05*c(0,max(postDen$y)), xlab=parname[j-2], ylab="density")
}
dev.off()

# Plot 2D posterior by plotting samples
par(mfrow=c(1,1), mgp=c(2.0,0.8,0), mar=c(3.0,3.0,0.5,0.5), 
    oma=0.1*c(1,1,1,1))
plot(postSamp[,3], postSamp[,4], xlab=parname[1], ylab=parname[2], pch=".")

# Given the samples drawn FROM the posterior PDF, it's easy to compute the
# summary statistics. We don't need the value of the posterior density anymore.
cat(mean(postSamp[,3]), mean(postSamp[,4]), sd(postSamp[,3]), sd(postSamp[,4]), 
    cor(postSamp[,3], postSamp[,4]), "\n")


Warning message in file(filename, "r", encoding = encoding):
“cannot open file 'metropolis.R': No such file or directory”

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection
